In [ ]:
from landlab import RasterModelGrid
from landlab.components import GroundwaterDupuitPercolator


In [ ]:
#initialize the grid and component

In [ ]:
grid = RasterModelGrid((10, 10), xy_spacing=10.0)
elev = grid.add_zeros("topographic__elevation", at="node")
abe = grid.add_zeros("aquifer_base__elevation", at="node")
elev[:] = 5.0
gdp = GroundwaterDupuitPercolator(grid)

In [ ]:
dt = 1E4
